<a href="https://colab.research.google.com/github/TsundereGus/ibd/blob/master/ibd-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Trabalho Prático 1

In [3]:
import io
import sqlite3
import pandas as pd

In [4]:
!wget --quiet -P /tmp/ https://raw.githubusercontent.com/TsundereGus/ibd/master/despesas.sql

In [5]:
conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

f = io.open('/tmp/despesas.sql', 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

##Questões da Seção 4.1 a serem resolvidas


2. Liste o valor total de despesas públicas por função (nome) e subfunção (nome), apenas
nos casos em que esse valor total excedeu 200 mil reais.


In [7]:
query = """
SELECT
  NOME_FUNCAO, NOME_SUBFUNCAO
FROM
  FUNCAO AS F, SUBFUNCAO AS S
WHERE
  F.COD_FUNCAO IN 
  (SELECT
    DET.COD_FUNCAO
  FROM
    DETALHAMENTO_FUNCAO AS DET
  WHERE
    F.COD_FUNCAO = DET.COD_FUNCAO
  AND
    S.COD_SUBFUNCAO = DET.COD_SUBFUNCAO
  AND
    DET.COD_DETALHAMENTO IN 
    (SELECT
      DESP.COD_DETALHAMENTO
    FROM
      DESPESA_PUBLICA AS DESP
    WHERE
      DET.COD_DETALHAMENTO = DESP.COD_DETALHAMENTO
    AND
      DESP.VALOR_DESPESA > 200000))
"""

df = pd.read_sql_query(query, conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Assistência social,Assistência comunitária
1,Educação,Educação básica


3. Liste todos os nomes dos órgaos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [8]:
query = """
SELECT
  O.NOME_ORGAO, SUM(DESP.VALOR_DESPESA) AS DESPESAS_2018
FROM
  ORGAO AS O, DESPESA_PUBLICA AS DESP
WHERE
  O.COD_ORGAO IN
  (SELECT
    DESP.COD_ORGAO FROM DESPESA_PUBLICA
  WHERE
    O.COD_ORGAO = DESP.COD_ORGAO
  AND
    ANO_DESPESA = 2018)
GROUP BY
  O.COD_ORGAO
ORDER BY
  SUM(DESP.VALOR_DESPESA) DESC
"""

df = pd.read_sql_query(query, conn)
df

,NOME_ORGAO,DESPESAS_2018
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,Presidência da República,0.00
6,"Ministério da Ciência, Tecnologia, Inovações",0.00
7,Ministério da Fazenda,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


4. Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do
órgão Ministério da Fazenda no primeiro semestre de 2018.

In [36]:
query = """
SELECT
  MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
FROM
  DESPESA_PUBLICA AS DESP, PROGRAMA_GOVERNO AS P 
WHERE
  MES_DESPESA IN (1, 2, 3)
AND
  COD_ORGAO = 25000
AND
  DESP.COD_PROGRAMA_GOVERNO = P.COD_PROGRAMA_GOVERNO
ORDER BY
  VALOR_DESPESA DESC
"""

df = pd.read_sql_query(query, conn)
df

,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
1,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


5. Liste o código e nome dos programas de governo que possuiram os 3 maiores valores
de despesas totais na função de educação durante o ano de 2018.

In [10]:
query = """
SELECT
  P.COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO 
FROM
  PROGRAMA_GOVERNO AS P, DESPESA_PUBLICA AS D 
WHERE
  D.COD_DETALHAMENTO IN (18, 19, 20, 21, 22)
AND
  ANO_DESPESA = 2018
GROUP BY 
  NOME_PROGRAMA_GOVERNO
LIMIT 3
"""

df = pd.read_sql_query(query, conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,24,BRASIL SEM MISERIA
1,18,CADEIAS PUBLICAS
2,8,CIENCIA SEM FRONTEIRAS


8. Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de
governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da
despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [11]:
query = """
SELECT
  P.NOME_PROGRAMA_GOVERNO, S.NOME_SUBFUNCAO, COUNT(DESP.COD_PROGRAMA_GOVERNO), SUM(DESP.VALOR_DESPESA)
FROM 
  PROGRAMA_GOVERNO AS P, SUBFUNCAO AS S, DESPESA_PUBLICA AS DESP
WHERE
  S.COD_SUBFUNCAO IN 
  (SELECT
    DET.COD_SUBFUNCAO
  FROM 
    DETALHAMENTO_FUNCAO AS DET 
  WHERE
    S.COD_SUBFUNCAO = DET.COD_SUBFUNCAO
  AND
    DET.COD_DETALHAMENTO IN 
    (SELECT
      DESP.COD_DETALHAMENTO
    FROM
      DESPESA_PUBLICA
    WHERE
      DET.COD_DETALHAMENTO = DESP.COD_DETALHAMENTO))
  AND
    P.COD_PROGRAMA_GOVERNO IN
    (SELECT 
      DESP.COD_PROGRAMA_GOVERNO
    FROM
      DESPESA_PUBLICA
    WHERE
      P.COD_PROGRAMA_GOVERNO = DESP.COD_PROGRAMA_GOVERNO)
GROUP BY
  P.NOME_PROGRAMA_GOVERNO, DESP.COD_PROGRAMA_GOVERNO, DESP.COD_DETALHAMENTO
"""



df = pd.read_sql_query(query, conn)
df

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,COUNT(DESP.COD_PROGRAMA_GOVERNO),SUM(DESP.VALOR_DESPESA)
0,BRASIL SEM MISERIA,Tecnologia da informação,26,58918.28
1,BRASIL SEM MISERIA,Comunicação social,1,0.00
2,BRASIL SEM MISERIA,Assistência comunitária,102,227883.77
3,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
4,BRASIL SEM MISERIA,Educação infantil,2,0.00
...,...,...,...,...
61,VIVER SEM LIMITE,Educação básica,1,0.00
62,VIVER SEM LIMITE,Transferências para a educação básica,2,0.00
63,VIVER SEM LIMITE,Assistência ao portador de deficiência,5,0.00
64,VIVER SEM LIMITE,Desenvolvimento científico,8,0.00


9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade,
valor) em que o órgão responsável é aquele que possui o maior número de órgãos
subordinados.

In [12]:
query = """
SELECT
  COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA
FROM
  DESPESA_PUBLICA AS DESP WHERE DESP.COD_ORGAO IN 
(SELECT
  O.COD_ORGAO
FROM
  ORGAO AS O
WHERE
  DESP.COD_ORGAO = O.COD_ORGAO 
AND
  O.QTD_ORGAOS_SUBORDINADOS = 104)
ORDER BY
  VALOR_DESPESA
"""


df = pd.read_sql_query(query, conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,196,10,2018,Reserva de Contingência,0.00
1,197,1,2018,Reserva de Contingência,0.00
2,198,2,2018,Reserva de Contingência,0.00
3,199,3,2018,Reserva de Contingência,0.00
4,200,4,2018,Reserva de Contingência,0.00
...,...,...,...,...,...
289,485,8,2018,Reserva de Contingência,2400.00
290,341,1,2018,APLICACOES DIRETAS - OPER.INTRA-ORCAMENTARIAS,5000.00
291,364,12,2018,Reserva de Contingência,5900.00
292,356,4,2018,Reserva de Contingência,18178.00


##Questões da Seção 4.2 a serem resolvidas

12

In [35]:
query = """
SELECT
  F.NOME_FUNCAO, S.NOME_SUBFUNCAO
FROM
  FUNCAO AS F, SUBFUNCAO AS S, DETALHAMENTO_FUNCAO AS DET
WHERE
  DET.COD_FUNCAO = F.COD_FUNCAO
AND
  DET.COD_SUBFUNCAO = S.COD_SUBFUNCAO
"""

df = pd.read_sql_query(query, conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


13

In [43]:
query = """
SELECT
  MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO
FROM
 DESPESA_PUBLICA AS DESP
INNER JOIN
  PROGRAMA_GOVERNO AS P ON P.COD_PROGRAMA_GOVERNO = DESP.COD_PROGRAMA_GOVERNO
WHERE
  MODALIDADE_DESPESA = 'Reserva de Contingência'
"""

df = pd.read_sql_query(query, conn)
df

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
...,...,...,...
671,Reserva de Contingência,0.0,BRASIL SEM MISERIA
672,Reserva de Contingência,0.0,BRASIL SEM MISERIA
673,Reserva de Contingência,0.0,BRASIL SEM MISERIA
674,Reserva de Contingência,0.0,BRASIL SEM MISERIA


14

In [41]:
query = """
SELECT
  VALOR_DESPESA, NOME_ORGAO
FROM
  DESPESA_PUBLICA AS DESP
INNER JOIN
  ORGAO AS O ON O.COD_ORGAO = DESP.COD_ORGAO
WHERE
  NOME_ORGAO = 'Ministério da Fazenda'
"""

df = pd.read_sql_query(query, conn)
df

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


15

In [23]:
query = """
SELECT
  NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
FROM
  DESPESA_PUBLICA AS DESP
INNER JOIN
  PROGRAMA_GOVERNO AS P ON P.COD_PROGRAMA_GOVERNO = DESP.COD_PROGRAMA_GOVERNO
INNER JOIN
  ORGAO AS O ON O.COD_ORGAO = DESP.COD_ORGAO
WHERE
  NOME_ORGAO = 'Ministério da Saúde'
"""

df = pd.read_sql_query(query, conn)
df

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
...,...,...
214,MAIS MEDICOS,0.0
215,MAIS MEDICOS,0.0
216,MAIS MEDICOS,0.0
217,MAIS MEDICOS,0.0
